# Configuration

> Configuration dataclass for single-file transcription workflow

In [ ]:
#| default_exp core.config

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from pathlib import Path
from dataclasses import dataclass, field
from typing import Optional, List

from cjm_fasthtml_workflow_transcription_single_file.core.html_ids import SingleFileHtmlIds
from cjm_fasthtml_workflow_transcription_single_file.media.config import MediaConfig
from cjm_fasthtml_workflow_transcription_single_file.storage.config import StorageConfig

## SingleFileWorkflowConfig Class

The `SingleFileWorkflowConfig` class provides configuration for the single-file transcription workflow. It includes settings for:
- Workflow identification and routing
- Worker process configuration
- UI customization options
- Media browser configuration
- Result storage configuration

In [ ]:
#| export
# Default config directory for workflow settings persistence
DEFAULT_WORKFLOW_CONFIG_DIR = Path("configs/workflows/single_file")

In [ ]:
#| export
@dataclass
class SingleFileWorkflowConfig:
    """Configuration for single-file transcription workflow."""

    # Workflow identification
    workflow_id: str = "single_file_transcription"  # Unique identifier for this workflow
    worker_type: str = "transcription:single_file"  # Worker process type identifier

    # Route configuration
    route_prefix: str = "/single_file"  # Base URL prefix for workflow routes
    stepflow_prefix: str = "/flow"  # Sub-prefix for StepFlow routes
    media_prefix: str = "/media"  # Sub-prefix for media browser routes

    # Container ID for workflow content
    container_id: str = SingleFileHtmlIds.WORKFLOW_CONTAINER  # HTML ID for main workflow container

    # UI options
    show_progress: bool = True  # Show step progress indicator
    max_files_displayed: int = 50  # Maximum files to show in simple file selector

    # Export formats supported
    export_formats: List[str] = field(default_factory=lambda: ["txt", "srt", "vtt"])  # Available export formats

    # Optional navigation URLs
    no_plugins_redirect: Optional[str] = None  # URL to redirect when no plugins configured
    no_files_redirect: Optional[str] = None  # URL to redirect when no media files found

    # SSE polling interval
    sse_poll_interval: float = 2.0  # Seconds between SSE status checks

    # Resource management
    gpu_memory_threshold_percent: float = 45.0  # Max GPU memory % before blocking new jobs

    # Configuration persistence
    config_dir: Path = field(default_factory=lambda: DEFAULT_WORKFLOW_CONFIG_DIR)  # Directory for workflow settings
    plugin_config_dir: Path = field(default_factory=lambda: DEFAULT_WORKFLOW_CONFIG_DIR / "plugins")  # Directory for plugin configs

    # Plugin category
    plugin_category: str = "transcription"  # Plugin category for this workflow

    # Internal subsystem configurations
    media: MediaConfig = field(default_factory=MediaConfig)  # Media scanning and display settings
    storage: StorageConfig = field(default_factory=StorageConfig)  # Result storage settings

    def get_full_stepflow_prefix(self) -> str:  # Combined route_prefix + stepflow_prefix
        """Get the full prefix for the StepFlow router."""
        return f"{self.route_prefix}{self.stepflow_prefix}"

    def get_full_media_prefix(self) -> str:  # Combined route_prefix + media_prefix
        """Get the full prefix for the media router."""
        return f"{self.route_prefix}{self.media_prefix}"

## Usage Examples

In [ ]:
# Create default configuration
config = SingleFileWorkflowConfig()
print(f"Workflow ID: {config.workflow_id}")
print(f"Route prefix: {config.route_prefix}")
print(f"Full stepflow prefix: {config.get_full_stepflow_prefix()}")
print(f"Full media prefix: {config.get_full_media_prefix()}")

Workflow ID: single_file_transcription
Route prefix: /single_file
Full stepflow prefix: /single_file/flow
Full media prefix: /single_file/media


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()